## Imports

In [2]:
from lz78 import Sequence, LZ78Encoder, CharacterMap, BlockLZ78Encoder, LZ78SPA
from lz78 import encoded_sequence_from_bytes, spa_from_bytes
import numpy as np
import lorem
import requests
from sys import stdout
from os import makedirs

## 3. LZ78 Sequential Probability Assignment (SPA)
The `LZ78SPA` class is the implementation of the family of sequential probability assignments discussed in [A Family of LZ78-based Universal Sequential Probability Assignments](https://arxiv.org/abs/2410.06589), for Dirichelt priors.
In this section, `gamma` refers to the Dirichlet parameter.

Under this prior, the sequential probability assignment is an additive
perturbation of the emprical distribution, conditioned on the LZ78 prefix
of each symbol (i.e., the probability model is proportional to the
number of times each node of the LZ78 tree has been visited, plus gamma).

This SPA has the following capabilities:
- training on one or more sequences,
- log loss ("perplexity") computation for test sequences,
- SPA computation (using the LZ78 context reached at the end of parsing
    the last training block),
- sequence generation.

Note that the LZ78SPA does not perform compression; you would have to use
a separate BlockLZ78Encoder object to perform block-wise compression.

### 3.1 Example: LZ78 SPA on Markov Data

We will use the Markov probability source used in [(Rajaraman et al, 2024)](https://arxiv.org/pdf/2404.08335), where the transition probability depends solely on $x_{t-k}$.
Specifically, $x_t = x_{t-k}$ with probability $0.9$, and otherwise $x_t$ is picked uniformly at random from the rest of the alphabet.

The SPA works best when the alphabet size is $2$, but you can try out other alphabet sizes too.

First, we define some helper functions for generating the data (don't worry about understanding these; they are irrelevant to understanding the SPA itself).

In [3]:
# Helper methods for generating data; feel free run the cell without
# reading the code
def sample_index_from_dist(probabilities):
    cdf = np.cumsum(probabilities)
    cdf[-1] = 1 # in case of FP error
    return int(np.where(np.random.random() < cdf)[0][0])

def entropy(probs):
    return sum([-x * np.log2(x) for x in probs if x > 0])

def get_stationary_dist(transition_probabilities):
    eigvals, eigvecs = np.linalg.eig(transition_probabilities.T)
    # all eigenvalues will be <= 1, and one will be =1
    stationary_dist = eigvecs[:, np.argmax(eigvals)]
    return stationary_dist / sum(stationary_dist)

def entropy_rate(transition_probabilities):
    stationary_dist = get_stationary_dist(transition_probabilities)
    return sum([prob * entropy(transition_probabilities[i]) 
                for i, prob in enumerate(stationary_dist)])

Generate some data to pass through the SPA:

In [4]:
## You can change these
ALPHABET_SIZE = 2
PEAK_PROB = 0.9
K = 5
N = 1_000_000
N_TEST = 10_000

In [5]:
# Build data array; feel free to ignore this code and just run the cell
transition_probabilities = np.eye(ALPHABET_SIZE) * PEAK_PROB + \
    (np.ones((ALPHABET_SIZE, ALPHABET_SIZE)) - np.eye(ALPHABET_SIZE)) * (1 - PEAK_PROB) / (ALPHABET_SIZE - 1)
start_prob = np.ones(ALPHABET_SIZE) / ALPHABET_SIZE

data = np.zeros(N, dtype=int)
for i in range(K):
    data[i] = sample_index_from_dist(start_prob)
for i in range(K,N):
    data[i] = sample_index_from_dist(transition_probabilities[data[i-K]])

In [6]:
sequence = Sequence(data[:-N_TEST], alphabet_size=ALPHABET_SIZE)

#### Instance method: `train_on_block`

Use a block of data to update the SPA. If `include_prev_context` is
true, then this block is considered to be from the same sequence as
the previous. Otherwise, it is assumed to be a separate sequence, and
we return to the root of the LZ78 prefix tree.

It returns the self-entropy log loss incurred while processing this
sequence.

In [7]:
spa = LZ78SPA(ALPHABET_SIZE)

In [8]:
stdout.flush()
spa.train_on_block(sequence) / (N - N_TEST)

0.7434136165483234

#### Instance method: `compute_test_loss`
After training a SPA, you can compute the log loss of a test sequence.

In [9]:
stdout.flush()
spa.compute_test_loss(Sequence(data[-N_TEST:], alphabet_size=ALPHABET_SIZE)) / N_TEST

0.6337562612642951

#### Instance method: `get_normalized_log_loss`
Gets the normaliized self-entropy log loss incurred from training the SPA thus far.

In [10]:
spa.get_normalized_log_loss()

0.7434136165483234

#### Instance method: `compute_spa_at_current_state`
Computes the SPA for every symbol in the alphabet, using the LZ78 context reached at the end of parsing the last training block.

In this case, the method will return a two-element list, where the first element is the estimated probability that the next symbol is $0$ and the second is the estimated probability that the next symbol is $1$.

In [11]:
# One component "should" be 0.9 and the other "should" be 0.1, but this is
# not necessarily the case. e.g., if we are at the top of the LZ78 prefix tree
# or at a leaf, we can expect the SPA to be closer to [0.5, 0.5]
spa.compute_spa_at_current_state()

[0.07110609480812641, 0.9288939051918735]

#### Instance method: `to_bytes`
This works the same as the corresponding index of `CompressedSequence`; refer to the LZ78 Encoding part of the tutorial for more details.

The method `spa_from_bytes` reconstructs a SPA from its `bytes` representation.

### Instance method: `reset_state`

Reset the state of the LZ78 prefix tree to the root. This can be called,
e.g., between training on two sequences that should be treated as separate
sequences.

This replaces the parameter `include_prev_context` for the SPA training and test functions in the first iteration of this library.

### 3.2 Example: Text Generation

Let's use the LZ78 SPA to generate some text based on Sherlock Holmes novels.

This requires the `requests` library and an internet connection.
If you don't have either, you can perform the same experiment any text you'd like, including the lorem ipsum text from the beginning of this tutorial.
Just make sure you have enough training data (e.g., the Sherlock novel used for this example is 500 kB).

In [1]:
from lz78 import Sequence, LZ78Encoder, CharacterMap, BlockLZ78Encoder, LZ78SPA
from lz78 import encoded_sequence_from_bytes, spa_from_bytes
import numpy as np
import lorem
import requests
from sys import stdout
from os import makedirs

In [2]:
text = requests.get("https://www.gutenberg.org/cache/epub/1661/pg1661.txt").text

Let's define our own character map and filter the text based on this.

In [3]:
stdout.flush()
charmap = CharacterMap("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ. ,?\n\"';:\t-_")
filtered_text = charmap.filter_string(text)

Next, train the SPA.

In [42]:
stdout.flush()
spa = LZ78SPA(charmap.alphabet_size(), gamma=0.2)
spa.train_on_block(Sequence(filtered_text, charmap=charmap)) / len(filtered_text)

3.784460934046932

In [53]:
spa.set_inference_params(
    gamma = 0.2,
    lb=1e-5,
    lb_or_temp_first="lb_first",
    ensemble_type="depth",
    ensemble_n=6,
    backshift_parsing=True,
    backshift_ctx_len=6,
    backshift_min_count=0,
)

In [54]:
stdout.flush()
spa.reset_state()
spa.compute_test_loss(Sequence(filtered_text, charmap=charmap)) / len(filtered_text)

2.995425013825895

#### Instance method: `generate_data`
Generates a sequence of data, using temperature and top-k sampling (see
the "Experiments" section of [Sagan and Weissman 2024] for more details).

Inputs:
- **len**: number of symbols to generate
- **seed_data**: you can specify that the sequence of generated data
    be the continuation of the specified sequence.
- **temperature**: a measure of how "random" the generated sequence is. A
    temperature of 0 deterministically generates the most likely
    symbols, and a temperature of 1 samples directly from the SPA.
    Temperature values around 0.1 or 0.2 function well.
- **top_k**: forces the generated symbols to be of the top_k most likely
    symbols at each timestep.
- **desired_context_length**: the SPA tries to maintain a context of at least a
    certain length at all times. So, when we reach a leaf of the LZ78
    prefix tree, we try traversing the tree with different suffixes of
    the generated sequence until we get a sufficiently long context
    for the next symbol.
- **min_spa_training_points**: requires that a node of the LZ78 prefix tree
    has been visited at least this number of times during training before
    it can be used for generation. i.e., instead of returning to the
    root upon reaching a leaf, we would return to the root once we reach
    any node that has not been traversed enough times.

Returns a tuple of the generated sequence and that sequence's log loss,
or perplexity.

Errors if the SPA has not been trained so far, or if the seed data is
not over the same alphabet as the training data.

In [94]:
spa.set_generation_params(
    gamma = 0.2,
    ensemble_type="disabled",
    # ensemble_n=6,
    backshift_parsing=True,
    backshift_ctx_len=20,
    backshift_min_count=1,
)

In [95]:
(generated, loss) = spa.generate_data(
    500,
    seed_data=Sequence("This ", charmap=charmap),
    temperature=0.2,
    top_k=5,
)
generated = generated.get_data()
generated = "This " + generated

In [96]:
def pretty_print(generated, max_line_width=80):
    for line in generated.split("\n"):
        line_width = 0
        for line2 in line.split("\t"):
            for word in line2.split(" "):
                if line_width + len(word) + 1 > max_line_width:
                    line_width = len(word)
                    print()
                    while line_width > max_line_width:
                        line_width -= max_line_width
                        print(word[:max_line_width])
                        word = word[max_line_width]                        
                else:
                    line_width += len(word)
                print(word, end=" ")
            if line_width + 8 <= max_line_width:
                line_width += 8
                print("\t", end="")
        print()
                

In [97]:
pretty_print(generated)

This is money was morning little to _e and a should be exposure is an orprise, and the thing 
behind he shot distribution is a man with a week a little daughter client he has dressed, and his 
who has we dried the very business as amust comply 	
heard a little dark profession. He was sob in the sum from the road is forced by the ashed back 
turned over. 	
 	
Well, it is 	
nally expense 	
to the stablish could not the sitting red 	
with the office been so that I feel to find a take a little George But if you would n 	
